In [ ]:
import triton
import triton.language as tl


@triton.jit 
def msefxn(predictions_ptr,targets_ptr,mse_ptr,N,BLOCK_SIZE:tl.constexpr):
    predictions_ptr = predictions_ptr.to(tl.pointer_type(tl.float32))
    targets_ptr = targets_ptr.to(tl.pointer_type(tl.float32))
    mse_ptr = mse_ptr.to(tl.pointer_type(tl.float32))
    pid = tl.program_id(0)
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    mask = offsets < N

    a = tl.load(predictions_ptr + offsets, mask=mask)
    b = tl.load(targets_ptr + offsets, mask=mask) 

    resdal=(b-a)*(b-a)
    val=tl.sum(resdal,axis=0)
    tl.atomic_add(mse_ptr, val/N)



def solve(predictions_ptr: int, targets_ptr: int, mse_ptr: int, N: int):
    BLOCK_SIZE = 256
    grid = (triton.cdiv(N, BLOCK_SIZE),)
    msefxn[grid](
        predictions_ptr, targets_ptr, mse_ptr, N,
        BLOCK_SIZE=BLOCK_SIZE
    )
    